# RDD

an RDD is the fundamental data structure of Apache Spark. It's a fault-tolerant, distributed collection of elements that can be operated on in parallel.

**Key Characteristics:**

- Immutable
- Lazy evaluation
- Fault tolerant (via lineage info)
- Partitioned across cluster nodes
- Can be cached in memory

### SparkContext and SparkConf


SparkContext is the entry point for Spark functionality.

#### `SparkConf`

- Configuration for Spark application

**Common settings:**

- setMaster("local[*]") – Use local mode with all cores
- setAppName("RDDExample") – Application name

e.g.,


### transformations

Transformations create a new RDD from an existing one. They are lazy – not executed until an action is triggered.

| Transformation  | Description                                          |
| --------------- | ---------------------------------------------------- |
| `map(func)`     | Returns a new RDD by applying `func` to each element |
| `filter(func)`  | Filters elements for which `func` returns true       |
| `flatMap(func)` | Like map but flattens the result                     |
| `distinct()`    | Removes duplicates                                   |
| `union(rdd)`    | Combines two RDDs                                    |
| `groupByKey()`  | Groups values with same key                          |
| `reduceByKey()` | Aggregates values with same key using a function     |
| `sortBy(func)`  | Sorts RDD by computed key                            |


### actions

Actions trigger computation and return results or write data.

| Action             | Description                            |
| ------------------ | -------------------------------------- |
| `collect()`        | Returns all elements to driver         |
| `count()`          | Returns number of elements             |
| `first()`          | Returns first element                  |
| `take(n)`          | Returns first `n` elements             |
| `reduce(func)`     | Reduces elements using binary operator |
| `saveAsTextFile()` | Writes RDD to text files               |




reference - [spark rdd docs](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


In [0]:
! pip install pyspark

# pyspark 4.0.0 - jdk 17 or above

In [0]:
! pip install pyspark==3.5.0

# pyspark 3.5 - jdk 11

In [0]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("regionSalesDemo").setMaster("local[*]")

sc = SparkContext(conf=conf)

In [0]:
sc.defaultParallelism

In [0]:
sc

In [0]:
sales_raw = sc.textFile("file:///workspaces/TRNG-2235-DB-Analyst/week1/datasets/region_sales_data.txt")

In [0]:
sales_raw.collect()

In [0]:
records = sales_raw.map(lambda x:x.split(","))

records.take(4)

In [0]:
records = records.map(lambda x:(int(x[0]), x[1], x[2], int(x[3])))
records.take(9)

In [0]:
# optional rdd- list
rdd = sc.parallelize([
    "2001,South,Ele,123",
    "2002,West,Furniture,345"
])

pairs = rdd.map(lambda line:[
    (line.split(",")[1], int(line.split(",")[3])),
    (line.split(",")[2], int(line.split(",")[3])),
])

pairs = pairs.flatMap(lambda x: x)


pairs.collect()

In [0]:
# total sales by category and region

category_sales = records.map(lambda x:((x[1]+x[2]), x[3]))
category_sales.collect()

In [0]:
# total sales by category

category_sales = records.map(lambda x:(x[2], x[3]))
category_sales.collect()

In [0]:
total_sales_by_category = category_sales.reduceByKey(lambda x,y:x+y)

In [0]:
total_sales_by_category.collect()

In [0]:
# average sales per category

average_sales_by_category = category_sales.mapValues(lambda x:(x,1)) \
                                .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
                                .mapValues(lambda x: x[0]/x[1])

average_sales_by_category.collect()

In [0]:
# highgest selling category

highest_selling_category = total_sales_by_category.max(key=lambda x: x[1])

highest_selling_category

In [0]:
# categories with sales above 12000

high_selling_cat_12k = total_sales_by_category.filter(lambda x: x[1]>12000)

high_selling_cat_12k.collect()

### Shared Variables

When you pass a function (like in map or reduce) to Spark, that function runs on different machines in the cluster — not on your local driver.

**By default:**

- Spark makes separate copies of any variable you use inside those functions.
- So, if a task changes a variable on the executor, that change will not reflect back in your driver program.

This is done to keep things fast and distributed — but it also means you can not just update normal variables across tasks.

**challenges:**

You want to count how many rows have Amount > 500 using this code:

```py
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[3]) > 500).foreach(increment_count)
print(count)

```

This will not work because each machine updates its own copy of count not the original one in the driver.

To solve such issues, spark provides two types of shared variables:

1. **Broadcast Variables:** a read-only variable that can be cached on each machine (executor). Used to efficiently share large data (like lookup tables) with all tasks without copying it multiple times.
2. **Accumulators:** variables used to safely implement counters or sums across mulitple worker nodes. You can only add to them (not read or subtract inside tasks). The final value is only accessible on the driver after an action is executed.




In [0]:
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[3]) > 500).foreach(increment_count)
print(count)

In [0]:
# broadcast variable

regions = {"North": "N", "South": "S", "East": "E", "West": "W"}

region_broadcast = sc.broadcast(regions)

In [0]:
# accumulator

high_value_count = sc.accumulator(0)

In [0]:
def enrich_region_and_count(record):
    pid, region, category, ammount = record
    if ammount > 800:
        high_value_count.add(1)
    return (pid, region_broadcast.value[region], category, ammount)

enriched_records = records.map(enrich_region_and_count)

enriched_records.collect()


In [0]:
enriched_records_2 = records.map(enrich_region_and_count)

enriched_records_2.collect()


In [0]:
high_value_count.value

In [0]:
# rdd to a new file

filtered = enriched_records.filter(lambda x:x[3]>800)

filtered.map(lambda x: ",".join(map(str,x))).repartition(2).saveAsTextFile("filtered_high_value_sales_3")

In [0]:
# rdd to a new file

filtered = enriched_records.filter(lambda x:x[3]>800)

filtered.map(lambda x: ",".join(map(str,x))).coalesce(1).saveAsTextFile("filtered_high_value_sales_2")

**Activity:**

1. find all product IDs where the amount is greater than 900.
2. Find all transactions that belong to the “Furniture” category.
3. Count how many transactions belong to the “Electronics” category.
4. Find average amount for each category.
5. Find the highest amount and the corresponding product ID.
6. Find the total number of unique categories.
7. For each category, find the product ID with the highest sale.
8. Count how many products were sold for less than 300.
9. Sort the transactions by amount in descending order.
10. Broadcast Variable: Category Discounts

```py
{"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}
```
11. Accumulator: Count Transactions Below 300
12. Filter and Save Results
